# Advent of code 2021

My first time entering Advent of Code. I'm pretty rusty in Python at the moment, so I have shameless copied the utility functions from Peter Norvieg's [2020 solutions](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) so that my attempts will at least start off in a well structured fashion.

Only entering to complete, not to get a fast time.

In [3]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, product, chain, tee
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union

import operator
import math
import ast
import sys
import re

Peter Norveigs utilities:

In [4]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms (numbers or strs), possibly ignoring a regex."
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Union[float, int, str]:
    "Parse text into a single float or int or str."
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text

My utilities

In [5]:
def pairwise(iterable): # Defined in itertools in python 3.10 but I'm in 3.9
    "pairwise('ABCDEFG') --> AB BC CD DE EF FG"
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

### Day 1
Part 1 is to compute the number of times we see an increase between elements. Itertools has a helpful _pairwise_ function that allows comparison between two adjacent elements in a list without making a full copy. Part 2 is to compute a sliding window first.. so I 'adapted' the _pairwise_ function to allow a 3-element sliding window but it's not nicely generalised to different window lengths.

In [6]:
in1: List[int] = data(1, int)

In [7]:
def day1_1(nums):
    return sum(second > first for first,second in pairwise(nums))

In [8]:
def day1_2(nums):
    "Niave extension of itertools.pairwise"
    a, b, c = tee(nums, 3)
    next(b, None)
    next(c, None)
    next(c, None)
    moving_average = [x+y+z for x, y, z in zip(a, b, c)]
    return day1_1(moving_average)

In [9]:
assert day1_1([199, 200, 208, 210, 200, 207, 240, 269, 260, 263]) == 7
assert day1_2([199, 200, 208, 210, 200, 207, 240, 269, 260, 263]) == 5

In [10]:
do(1, 1832, 1858)

[1832, 1858]

### Day 2

For part 1 we don't need to worry about the order of the commands (assuming that we don't need to verify that the depth is always sensible). Brute fore approach for part 2.

In [11]:
Instruction = Tuple[str, int] # e.g. ('up', 1)
Program = List[Instruction]

in2: Program = data(2, atoms)

In [12]:
def sum_direction(program, direction):
    return sum(x[1] for x in filter(lambda x: x[0]==direction, program))

assert sum_direction([('down',2),('forward',7),('down',1)],"down") == 3

def day2_1(program):
    "Assumes that the input is valid and we don't have to worry about the submarine trying to go above the waterline"
    horizontal_position = sum_direction(program,"forward")
    depth = sum_direction(program,"down") - sum_direction(program,"up")
    return depth * horizontal_position

In [13]:
def day2_2(program):
    aim, depth, horizontal_position = 0, 0, 0
    for command, v in program:
        if command == "down":
            aim += v
        elif command == "up":
            aim -= v
        elif command == "forward":
            depth += aim * v
            horizontal_position += v
        else:
            raise Exception('Unexpected command %s',command)
    return depth * horizontal_position

In [14]:
do(2, 1484118, 1463827010)

[1484118, 1463827010]

### Day 3
Full brute force today. For part I I'm sure there must be a better way to unpack the most common element from a _Counter_ and to use the two's complement operator to compute epsilon. In part II, there must be a better way to find the most common element with a default.

I might revisit  these at a later date.

In [15]:
in3: List(int) = data(3, str)

In [16]:
def day3_1(nums):
    nums = [list(x) for x in nums]
    nums = list(map(list, zip(*nums)))
    counters = [Counter(x) for x in nums]
    gamma = "".join(str(x.most_common(1)[0][0]) for x in counters)
    epsilon = "".join(str(x.most_common()[-1][0]) for x in counters)
    return int(gamma,base=2) * int(epsilon, base=2)

def get_rating(nums, mode):
    ix = 0
    while len(nums) > 1 and ix < len(nums[0]):
        counter = Counter(x[ix] for x in nums)
        v = counter.most_common(1)[0] if mode=="most" else counter.most_common()[-1]
        if v[1]*2 == len(nums):
            v = '1' if mode=="most" else '0'
        else:
            v = v[0]
        nums = list(filter(lambda x: x[ix] == v,nums))
        ix += 1
    return nums[0]

def day3_2(nums):
    oxygen = get_rating(nums,mode="most")
    co2 = get_rating(nums,mode="least")
    return int(oxygen,base=2) * int(co2,base=2)

In [17]:
do(3, 2724524, 2775870)

[2724524, 2775870]

### Day 4
I implemented a BingoBoard class on my first attempt here, but decided this was unnecessary when I revisited it. I think my runtime complexity is _O(NDB^2)_ for N boards, B elements per board and D draws because of the _is_bingo_ function, but the input is not that big.

In [44]:
def parse_bingo_boards(line: str) -> List[int]:
    return list(map(int,re.split("\W+",line.strip())))

in4: List[int] = data(4, parse_bingo_boards, sep='\n\n')

In [37]:
def is_bingo(hits, width = 5):
    out = any(all(hits[i:i+width]) for i in range(0,len(hits),width)) # check for complete rows
    if not out:
        out = any(all(hits[i:len(hits):width]) for i in range(width)) # check for complete columns
    return out

def score_bingo_board(board, draws):
    hits = [False for x in range(len(board))]
    bingo = False
    for i, draw in enumerate(draws):
        if draw in board:
            hits[board.index(draw)] = True
            bingo = is_bingo(hits)
            if bingo: break
    if bingo:
        return (i, draw * sum(x for x, h in zip(board,hits) if not h))
    else:
        return (math.inf, None)

In [47]:
def day4_1(boards):
    scores = [score_bingo_board(x, boards[0]) for x in boards[1:]]
    out = min(scores, key = lambda x: x[0])
    return out[1]

def day4_2(boards):
    scores = [score_bingo_board(x, boards[0]) for x in boards[1:]]
    scores = filter(lambda x: math.isfinite(x[0]), scores)
    out = max(scores, key = lambda x: x[0])
    return out[1]

In [48]:
do(4, 49860, 24628)

[49860, 24628]